In [36]:
import numpy as np
import math


def truncation(function):

    def method(*args,accuracy=6):
        
        #python will automatically pass class object
        
        ans = function(*args)
        
        if isinstance(ans,dict):
            
            for key,value in ans.items():
                
                ans[key] = np.around(value,accuracy)
                
            return ans
        
        elif isinstance(ans,list):
            
            for i,value in enumerate(ans):
                
                ans[i] = np.around(value,accuracy)
                
            return ans

        return np.around(ans,accuracy)

    return method


# code and test voltage controle class on 28/11/2020

# discovered np.around could round both complex and flotting point numbers

# added trucate decorator and try to add trucate_dict to set precisions for key value pair in dictonary

In [49]:
class BasicElectrical:
    
    def __init__(self,accuracy=6
                 
                 ,freq=50):

        self.accuracy = accuracy

        self.freq = freq
        
        np.set_printoptions(precision=3)
    
    @truncation
    def power(self,V,I):

        return V*(I.conjugate())
    
    @truncation
    def PhaseDiff(self,a,b):
        
        if a.real!=0 and b.real!=0:
        
            alpha = np.arctan(a.imag/a.real)

            beta = np.arctan(b.imag/b.real)
            
        elif a.real==0 and b.real!=0:
            
            alpha = np.pi/2
            
            beta = np.arctan(b.imag/b.real)
        
        elif a.real!=0 and b.real==0:
            
            alpha = np.arctan(a.imag/a.real)
            
            beta = np.pi/2
        
        else:
            return 0
        
        return math.degrees(alpha-beta)
    
    @truncation
    def polar(self,r,theta):
        
        theta = math.radians(theta)
        
        x = round(r*np.cos(theta),self.accuracy)
        
        y = round(r*np.sin(theta),self.accuracy)
        
        return complex(x,y)
    
    def scientific_notation(self,params,precision=3):
        
        if isinstance(params,dict):
            
            formated_dict = dict()
            
            for key,value in params.items():
                
                if isinstance(value,complex):
                    
                    formated_dict[key] = f"{np.format_float_scientific(value.real,precision=precision)} + j{np.format_float_scientific(value.imag,precision=precision)}"
                
                else:
                    
                    formated_dict[key] = f"{np.format_float_scientific(value,precision=precision)}"

            return formated_dict
                    
        elif isinstance(params,list):
            
            formated_list = list()
            
            for param in params:
                
                if isinstance(param,complex):

                    formated_list.append(f"{np.format_float_scientific(param.real,precision=precision)} + j{np.format_float_scientific(param.imag,precision=precision)}")

                else:

                    formated_list.append(f"{np.format_float_scientific(param,precision=precision)}")
            
            return formated_list
                    
        else:
            
            if isinstance(params,complex):

                print(f"{np.format_float_scientific(params.real,precision=precision)} + j{np.format_float_scientific(params.imag,precision=precision)}")
            
            else:
        
                print(f"{np.format_float_scientific(params,precision=precision)}")
            
    @truncation   
    def propagation_constants(self,z,y,length):
        
        assert length>500 "long transmission line length should be grater than 500k"
        
        gama = np.sqrt(z*y)
        
        alpha = gama.real
        
        beta = gama.imag
        
        Zc = np.sqrt(z/y)
        
        params = {
            'gama':gama,
            
            'alpha':alpha,
            
            'beta':beta,
            
            'SurgeImpedence':Zc
        }
        
        return params
        
    def ABCD_T(self,Z,Y):
        
        A = 1+(Y*Z)/2 ; B= Z*(1+(Y*Z)/4) ; C = Y ; D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_T(self,Vr,Ir,Z,Y):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_T(Z,Y)
        
        return np.matmul(A*R)
    
    def ABCD_PI(self,Z,Y):
        
        A = 1+(Y*Z)/2 ; B=Z ; C=Y(1+(Y*z)/4); D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_PI(self,Vr,Ir,Z,Y):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_PI(Z,Y)
        
        return np.matmul(A*R)
    
    def ABCD_Long(self,z,y,length):
        
        gama = pow(z*y,0.5)
        
        Zc = pow(z/y,0.5)
        
        A = np.cosh(gama*length) ; B = Zc*np.sinh(gama*length) ; C = np.sinh(gama*length)/Zc ; D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_Long(self,Vr,Ir,z,y,length):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_Long(z,y,length)
        
        return np.matmul(A*R)
    
    @truncation
    def PhaseComponents(self,Va0,Va1,Va2):
        
        Lambda = self.polar(1,120)

        Va = Va0+Va1+Va2

        Vb = Va0+Lambda*Lambda*Va1+Lambda*Va2

        Vc = Va0+Lambda*Va1+Lambda*Lambda*Va2

        V = [Va,Vb,Vc]

        return V
    
    @truncation
    def SymetricComponents(self,Va,Vb,Vc):
        
        Lambda = self.polar(1,120)

        Va0 = (1/3)*(Va+Vb+Vc)

        Va1 = (1/3)*(Va+Lambda*Vb+Lambda*Lambda*Vc)

        Va2 = (1/3)*(Va+Lambda*Lambda*Vb+Lambda*Vc)

        V = [Va0,Va1,Va2]

        return list(V)
    
    def convert_star_to_delta(self,z1,z2,z3):
        
        za = (z1*z2)/(z1 + z2 + z3)
        
        zb = (z2*z3)/(z1 + z2 +z3)
        
        zc = (z3*z1)/(z1 + z2 +z3)
        
        return za, zb, zc
    
    def convert_delta_to_star(self,za,zb,zc):
        
        z1 = (za + zb + (za*zb)/zc)
        
        z2 = (zb + zc + (zb*zc)/za)
        
        z3 = (za + zc + (za*zc)/zb)
        
        return z1, z2, z3

In [50]:
class VoltageControle(BasicElectrical):
    
    def __init__(accuracy=6):

        super().__init__(accuracy=accuracy)
        
    
    @truncation
    def SynchronousMachine(self,E,V,X,delta):
        
        delta = math.radians(delta)
        
        P = abs(E)*abs(V)*np.sin(delta)/-X
   
        Q = (abs(E)*abs(V)*np.cos(delta)/X) - (pow(abs(V),2)/X)
    
        return complex(P,Q)
    
    @truncation
    def TapChaningTransformer(self,V1,V2,P,Q,R,X):
        
        ans = (V2/V1)*(1/(1-(R*P+X*Q)/(V1*V2)))
        
        return pow(ans,0.5)
    
    
    def CircleParams(self,Vs,Vr,A,B,flag=False):
        
        beta = np.arctan(B.Imag/B.real)
        
        alpha = np.arctan(A.Imag/A.real)
        
        r = pow(abs(Vs)*abs(Vr)/abs(B),2)
        
        if flag is False:
            
            D = abs(A)*pow(abs(Vs),2)/abs(B)
        
            x = D*np.cos(beta-alpha)
        
            y = D*np.sin(beta-alpha)
        
            print(f'Center (x,y) => {x,y} radius => {r}')
            
            print(f'Distance B/w origin and center {D} at an angle {math.radians(beta-alpha)}')
            
            return x,y,r
            
        else:
            
            D = abs(A)*pow(abs(Vr),2)/abs(B)
        
            x = D*np.cos(beta-alpha)
        
            y = D*np.sin(beta-alpha)
            
            print(f'Center (x,y) => {-x,-y} radius => {r}')
            
            print(f'Distance B/w origin and center {D} at an angle {math.radians(beta-alpha)}')
            
            return -x,-y,r
    
    def SendingEndCircle(self,Vs,Vr,D,B):
        
        return self.CircleParams(Vs,Vr,D,B,flag=False)
    
    def ReceiveEndCircle(self,Vs,Vr,A,B):
        
        return self.CircleParams(Vs,Vr,A,B,flag=True)
        

In [51]:
class Faults(BasicElectrical):
    
    def __init__(self,Ea,z0=None,z1=None,z2=None,zn=None,zf=None,accuracy=3):
        
        super().__init__(accuracy=accuracy)
        
        self.Ea = Ea
        
        self.z0 = z0
        
        self.z1 = z1
        
        self.z2 = z2
        
        self.zn = zn
        
        self.zf = zf
        
    @truncation    
    def FaultData(self,Va0,Va1,Va2,Ia0,Ia1,Ia2):

        [Va,Vb,Vc] = self.PhaseComponents(Va0,Va1,Va2)

        [Ia,Ib,Ic] = self.PhaseComponents(Ia0,Ia1,Ia2)

        data = {'Ia0':Ia0,'Ia1':Ia1,'Ia2':Ia2,
                'Va0':Va0,'Va1':Va1,'Va2':Va2,
                'Va':Va,'Vb':Vb,'Vc':Vc,
                'Ia':Ia,'Ib':Ib,'Ic':Ic}
        
        return data   
    
    def SingleLineFault(self):

        Ia1 = (self.Ea)/(self.z0+self.z1+self.z2)

        Ia0 = Ia1

        Ia2 = Ia1

        Va1 = self.Ea - Ia1*self.z1

        Va2 = -Ia2*self.z2

        Va0 = -Ia0*self.z0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        return FaultData

    def LineToLineFault(self):

        Ia0 = 0

        Ia1 = (self.Ea)/(self.z1+self.z2)

        Ia2 = -Ia1

        Va1 = self.Ea - Ia1*self.z1

        Va2 = -Ia2*self.z2

        Va0 = -Ia0*self.z0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)

        return FaultData

    def DoubleLineGroundFault(self):

        Ia1 = (self.Ea)/(self.z1+(self.z0*self.z2)/(self.z0+self.z2))

        Va1 = self.Ea - Ia1*self.z1

        Va0 = Va1

        Va2 = Va1

        Ia0 = -Va0/self.z0

        Ia2 = -Va2/self.z2
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        return FaultData

    def ThreePhaseFault(self):

        Ia0 = 0

        Ia1 = self.Ea/self.z1

        Ia2 = 0

        Va0 = 0

        Va1 = 0

        Va2 = 0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)

        return FaultData
    
    def SingleLineFaultZ(self):
    
        Ia1 = (self.Ea)/(self.z0+self.z1+self.z2+3*(self.zn+self.zf))
        
        Ia2 = Ia1
        
        Ia0 = Ia1
        
        Va0 = -Ia0*(self.z0+3*self.zn)
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        return FaultData

    def LineToLineFaultZ(self):

        Ia1 = (self.Ea)/(self.z1+self.z2+self.zf)
        
        Ia2 = -Ia1
        
        Ia0 = 0
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        Va0 = -Ia0*(self.z0)
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        return FaultData

    def DoubleLineFaultz(self):

        zeq = self.z2*(self.z0+3*self.zn+3*self.zf)/(self.z2+self.z0+3*self.zn+3*self.zf)

        Ia1 = (self.Ea)/(self.z1+zeq)
        
        Ia2 = -1*(self.Ea-self.Ia1*self.z1)/(self.z2)
        
        Ia0 = -1*(self.Ea-Ia1*self.z1)/(self.z0+3*(self.zn+self.zf))
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        Va0 = -Ia0*(self.z0+3*self.zn)
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)

        return FaultData

In [52]:
SymetricComponents = BasicElectrical().SymetricComponents

Va = complex(500,150)

Vb = complex(100,-600)

Vc = complex(-300,600)

A = SymetricComponents(Va,Vb,Vc)

BasicElectrical().scientific_notation(A)

['1.e+02 + j5.e+01', '5.464e+02 + j1.655e+02', '-1.464e+02 + j-6.547e+01']

In [53]:
SymetricComponents = BasicElectrical().SymetricComponents

polar = BasicElectrical().polar

Va = polar(100,0)

Vb = polar(33,-100)

Vc = polar(38,176.5)

A = SymetricComponents(Va,Vb,Vc)

BasicElectrical().scientific_notation(A)

['1.878e+01 + j-1.006e+01',
 '5.066e+01 + j1.432e+01',
 '3.056e+01 + j-4.265e+00']

In [54]:
z0 = complex(0,0.1)

z1 = complex(0,0.25)

z2 = complex(0,0.35)

Ea = 1

faults = Faults(Ea,z0,z1,z2)

FaultData = faults.DoubleLineGroundFault()


BasicElectrical().scientific_notation(FaultData)

{'Ia0': '-0.e+00 + j2.373e+00',
 'Ia1': '0.e+00 + j-3.051e+00',
 'Ia2': '-0.e+00 + j6.78e-01',
 'Va0': '2.373e-01 + j0.e+00',
 'Va1': '2.373e-01 + j0.e+00',
 'Va2': '2.373e-01 + j0.e+00',
 'Va': '7.119e-01 + j0.e+00',
 'Vb': '1.e-05 + j0.e+00',
 'Vc': '1.e-05 + j0.e+00',
 'Ia': '0.e+00 + j0.e+00',
 'Ib': '-3.229e+00 + j3.559e+00',
 'Ic': '3.229e+00 + j3.559e+00'}

In [55]:
E = BasicElectrical()

z = complex(0.5,0.9)

y = complex(0,np.float(6e-6))

A = E.propagation_constants(z,y,200)

BasicElectrical().scientific_notation(A)

{'gama': '6.23e-04 + j2.406e-03',
 'alpha': '6.23e-04',
 'beta': '2.406e-03',
 'SurgeImpedence': '4.01e+02 + j-1.039e+02'}

In [56]:
E.PhaseDiff(z,y)

-29.054604

In [57]:
V = VoltageControle()

E = polar(220,67)

v = polar(200,0)

delta = 37

X = 20

V.SynchronousMachine(E,v,X,delta)

(-1323.993052-243.001877j)

In [58]:
V = np.float(220)*10**3

I = np.complex(116,30)

BasicElectrical().power(V,I)

(25520000-6600000j)